In [1]:
import requests
from datasets import load_dataset
from tqdm import tqdm
import json
import dotenv
import os

In [2]:
dotenv.load_dotenv()
API_KEY = []
API_KEY.append(os.getenv("API_GPT"))

In [3]:
dataset_name = "kokujin/prompts_1"
model_name = "GPT4-o-mini"
output_dir = f"./results/{model_name}/"

headers = {"Content-Type": "application/json",
           "Authorization": f"Bearer {API_KEY[0]}"}

In [4]:
test = load_dataset(dataset_name, split="test")

opt = model_name
dic = {opt: []}

In [5]:
import logging

# Set up logging
log_filename = f"logsgpt.txt"
logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Example log messages
logging.info("Started processing the dataset.")
logging.info(f"Dataset name: {dataset_name}")
logging.info(f"Model name: {model_name}")
logging.info("Finished processing the dataset.")

In [6]:
import time

start = time.time()
opt_ = 0
for i, prompt in tqdm(enumerate(test)):
    
    url = f'https://api.openai.com/v1/chat/completions'

    tmp = {
        "Prompt": "",
        "Original": "",
        "Prediction": ""
    }
    resp = prompt["Text"].split('### Output: ')
    prompt = resp[0]
    payload = {"model": "gpt-4o-mini","messages": [{"role": "system", "content": prompt}]}
    #print(payload)

    res = requests.post(url, json=payload, headers=headers)
    logging.info(f"Prompt: {res}")
    resp_ = json.loads(res.text)
    logging.info(f"Prompt: {resp_}")
    #print(resp_['candidates'][0]['content']['parts'][0]['text'])
    result = resp_['choices'][0]['message']['content']
    tmp["Prompt"] = prompt
    tmp["Original"] = resp[1]
    tmp["Prediction"] = result
    dic[opt].append(tmp)

2399it [3:37:17,  5.43s/it]


In [7]:
results = {
    "Prompt": [],
    "Original": [],
    "Prediction": []
}

In [8]:
def format(val):
    val = val.replace("'", '"')
    #    val = '"' + val + '"'
    return val

In [9]:

ommited = []
a = []
for i, t in enumerate(dic[opt]):
    try:
        val_p = t["Prediction"]

        val_p = json.loads("}".join(('{' + "{".join(val_p.split('{')[1:])).split('}')[:-1]) + '}')
        
        #print(val_p, '\n aca termina la review')
        #val_p = format(val_p)
        #val_p = val_p + "\"}" if "\"}" not in val_p else val_p

        val = t["Original"]
        val = format(val)
        val = json.loads(val)
        #print(val, '\n aca termina la review')
        results["Prediction"].append(val_p)
        results["Original"].append(val)
        results["Prompt"].append(t["Prompt"])

    
    except Exception as e:
        ommited.append(i)
        #print(e)
        a.append(t["Prediction"])

print(len(ommited), ommited)

158 [2, 9, 10, 21, 22, 35, 43, 64, 66, 75, 96, 107, 124, 150, 167, 194, 205, 228, 248, 250, 278, 279, 282, 304, 330, 339, 357, 360, 393, 405, 423, 444, 454, 462, 465, 470, 473, 479, 499, 534, 561, 585, 592, 599, 621, 633, 639, 648, 677, 693, 695, 707, 727, 729, 731, 776, 804, 844, 852, 859, 899, 903, 930, 946, 989, 997, 1038, 1043, 1070, 1084, 1117, 1160, 1164, 1167, 1205, 1212, 1214, 1229, 1252, 1282, 1309, 1335, 1352, 1370, 1382, 1402, 1431, 1441, 1450, 1456, 1466, 1481, 1487, 1494, 1508, 1520, 1525, 1528, 1530, 1539, 1543, 1550, 1568, 1577, 1580, 1584, 1610, 1619, 1632, 1635, 1636, 1659, 1660, 1661, 1686, 1698, 1782, 1819, 1836, 1860, 1862, 1868, 1893, 1917, 1921, 1934, 1959, 1966, 1984, 1991, 1998, 1999, 2010, 2011, 2058, 2079, 2081, 2092, 2094, 2102, 2114, 2151, 2183, 2194, 2227, 2263, 2265, 2308, 2324, 2329, 2345, 2346, 2348, 2352, 2379, 2382, 2384, 2389]


In [10]:
print(len(results['Prediction']))

2241


In [11]:
with open(f'Outputs/{model_name}/{model_name}.json', 'w') as f:
    json.dump(results, f, indent=4)